# TODO, basic mcp server setup for openai agent sdk

In [3]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os


load_dotenv()

True

# get a list of tools provided by MCPServerStiom

In [6]:
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]} #only spawn fetch tool

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    fetch_tools = await server.list_tools()
fetch_tools #then should only return one fetch tool here

[Tool(name='fetch', title=None, description='Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.', inputSchema={'description': 'Parameters for fetching a URL.', 'properties': {'url': {'description': 'URL to fetch', 'format': 'uri', 'minLength': 1, 'title': 'Url', 'type': 'string'}, 'max_length': {'default': 5000, 'description': 'Maximum number of characters to return.', 'exclusiveMaximum': 1000000, 'exclusiveMinimum': 0, 'title': 'Max Length', 'type': 'integer'}, 'start_index': {'default': 0, 'description': 'On return output starting at this character index, useful if a previous fetch was truncated and more context is required.', 'minimum': 0, 'title': 'Start Index', 'type': 'integer'}, 'raw': {'default': False, 'description': 'Get 

In [7]:
playwright_params = {"command": "npx", "args": ["@playwright/mcp@latest"]}
async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60)  as server:
    playwright_tools = await server.list_tools()

playwright_tools

[Tool(name='browser_close', title=None, description='Close the page', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, icons=None, annotations=ToolAnnotations(title='Close browser', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None, execution=None),
 Tool(name='browser_resize', title=None, description='Resize the browser window', inputSchema={'type': 'object', 'properties': {'width': {'type': 'number', 'description': 'Width of the browser window'}, 'height': {'type': 'number', 'description': 'Height of the browser window'}}, 'required': ['width', 'height'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, icons=None, annotations=ToolAnnotations(title='Resize browser window', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None, execution=None),

In [11]:
sandbox_path =  os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
os.makedirs(sandbox_path, exist_ok=True) #make the dir if not exist
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params, client_session_timeout_seconds=60) as server:
    file_tools = await server.list_tools()
file_tools

[Tool(name='read_file', title='Read File (Deprecated)', description='Read the complete contents of a file as text. DEPRECATED: Use read_text_file instead.', inputSchema={'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'properties': {'path': {'type': 'string'}, 'tail': {'description': 'If provided, returns only the last N lines of the file', 'type': 'number'}, 'head': {'description': 'If provided, returns only the first N lines of the file', 'type': 'number'}}, 'required': ['path']}, outputSchema={'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'properties': {'content': {'type': 'string'}}, 'required': ['content'], 'additionalProperties': False}, icons=None, annotations=ToolAnnotations(title=None, readOnlyHint=True, destructiveHint=None, idempotentHint=None, openWorldHint=None), meta=None, execution=ToolExecution(taskSupport='forbidden')),
 Tool(name='read_text_file', title='Read Text File', description="Read the complete contents of a fil

# That's a lots of tools (~30 tools or more), I am wondering if agent can actuall process this

In [20]:
instruction = """
You browser the internet to accomplish your instruction.
Browse independently, accept cookies, clicking "not now' or similar options to get the content you need.
If attempts on one websites is not fruitful, try another.
Try different options and sites as needed, 
come up with final answers by yourself. Only answer, do not ask back users for clarification or confirmation.
"""

message = "find a gathering events happen in nova, va area between dec 15 and dec 25 2025"

async with MCPServerStdio(params=files_params, client_session_timeout_seconds=60) as mcp_server_files:
    async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as mcp_server_browser:
        agent = Agent(
            name="investigator",
            instructions=instruction,
            model="gpt-5-mini",
            mcp_servers=[mcp_server_files, mcp_server_browser]

        )
        with trace("udemy-investigate"):
            result = await Runner.run(agent, message, max_turns=25)
            print (result.final_output)

Here are recurring/annual gatherings and holiday events in Northern Virginia that typically run during mid–late December and that are very likely to have dates between Dec 15–25, 2025. I couldn’t reliably scrape every official page right now, so please verify exact performance/dates & times on the organizer sites listed.

1) Meadowlark Botanical Gardens — Winter Walk of Lights (Vienna)  
   - Large outdoor light display open nightly through Dec/Jan.  
   - Where to check: NOVA Parks / Meadowlark Botanical Gardens events page.

2) George Washington’s Mount Vernon — “Mount Vernon by Candlelight” / holiday tours (Mount Vernon / Alexandria)  
   - Candlelit tours, living-history programs and other holiday evenings in December (selected nights).  
   - Where to check: mountvernon.org — Events / Tickets.

3) The Nutcracker and holiday ballet/concert performances (various venues: The Alden in McLean, community theaters, dance companies)  
   - Multiple productions run mid–late December across